## NLP (NLTK brown corpus)- Clustering & Nearest Neighbour

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re, string
import numpy
from sklearn.cluster import KMeans

In [3]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import text
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
%matplotlib inline

### Download the Brown corpus (using nltk.corpus)

Legends: 
  1. For clarity all the local variable names used in notebook are specified in braces. 
  2. For description of algorithm and logic flow semantic terms are used (with variable names in braces)
  
  Purpose : 
  For NLP or English language based application we are making an endeavour to have a clustering or embedding of these words. Basically similar-semantic meaning words will be grouped close to each other 
  
Input Data:  
Loading the Brown-corpus words from the "NLTK" library, and using the inbuilt function created a list of all the available words in local list. This list will contain stop words, punctutions etc.  

In [4]:
brwn_lst = brown.words()

In [5]:
print (len(brwn_lst))

1161192


### Removing stopwords and punctuation, make everything lowercase, and counting how often each word occurs.

1. Extended the inbuilt available "String Punctuations" list to include few extra punctions like '--' etc.
2.  Removed all the punctuations from list (words_lst)
3. Changed all words to lower case in list (words_lst)
4. Removed all the "ENGLISH_STOP_WORDS" & "nltk_stops" - as stop words don't contribute much to the sematic relevance when embedding a given words 

In [5]:
extras = ['``','--',"''"]
punctuations = extras + list(string.punctuation)

In [6]:
words_lst = [x for x in brwn_lst if x not in punctuations]
len(words_lst)

1013320

In [7]:
words_lst = [x.lower() for x in words_lst ]

In [8]:
words_lst = [x for x in words_lst if x not in text.ENGLISH_STOP_WORDS]
len(words_lst)

486942

In [9]:
nltk_stops = stopwords.words('english')
words_lst = [x for x in words_lst if x not in nltk_stops ]  #Removing nltk.corpus stopwords
len(words_lst)

483756

In [10]:
word_freq_dict = {}
for word in words_lst :
    word_freq_dict[word] = word_freq_dict.setdefault(word, 0) + 1

len(word_freq_dict)

49474

In [11]:
words_sorted_lst  = sorted(word_freq_dict.items(), key=lambda x:x[1], reverse=True)

In [14]:
V = [x[0] for x in words_sorted_lst[:5000] ]
#print (V[-10:])
C = V[:1000]
#print (C[-10:])

#### Vocubulary & Context words list creation 

Based on the usage/occurance frequency (high usage) of each word created a Vocubulary list (V) of around 5k words and Context list (C) of 1000 words. 

#### Constructing the probability distribution Pr(c|w) of con- text words around w (for each w ∈ V ), as well as the overall distribution Pr(c) of context words. 

In [15]:
N_WC = numpy.zeros(shape=(1000,5000))

In [16]:
#AL - Uncommment the following line for adding the smoothing one 
#N_WC[N_WC == 0] = 1

In [17]:
## AL - Padding list with 2 extra blank strings to addess boundary cases of words. 
print (len(words_lst))
words_lst.insert(0, "")
words_lst.insert(0, "")
words_lst.append("")
words_lst.append("")
print (len(words_lst))

483756
483760


In [18]:
for w in V:
    indices = [i for i, x in enumerate(words_lst) if x == w]
    word_idx = V.index(w)
    
    for idx in range(len(indices)):
        word_loc = indices[idx]
        w1 = words_lst[word_loc-2]
        w2 = words_lst[word_loc-1]
        w3 = words_lst[word_loc+1]
        w4 = words_lst[word_loc+2]
        w =  words_lst[word_loc]

        word_set = set([w1, w2, w3, w4])

        for elem in word_set:
            contxt_idx = -1
            try:
                contxt_idx = C.index(elem)
            except: 
                continue
            else:
                N_WC[contxt_idx,word_idx] += 1

In [19]:
cols_sum = N_WC.sum(axis=0)
print (len(cols_sum))
print (cols_sum)

5000
[ 3358.  2888.  2867. ...,    31.    44.    24.]


In [20]:
rows_sum = N_WC.sum(axis=1)
print (len(rows_sum))
print (rows_sum[:10])

1000
[ 5691.  4888.  4732.  3560.  3534.  2497.  2886.  2656.  2605.  2215.]


In [21]:
total_rows_sum = np.sum(rows_sum)
#total_rows_sum

568892.0

1.  Occurance of C in window W
2.  Padded the list V with two empty string to handle the boudary cases of four words window.
3.  Calculated the n(w, c) = # of times c occurs in a window around w in a Matrix form (numpy array of dim VxC as N_WC)
4. Using the count in each cell - construct the probability distribution Pr(c|w) of context words around w, for all words in V (Pr_CW).
5.  Also calculated the Probabilties of each word (say c) in C (Pr_C)





###  Represent each vocabulary item w by a |C|-dimensional vector φ(w), whose c’th coordinate is:
φ(w) = max(0, log Pr(c|w) ) Pr(c)
#### This is known as the (positive) pointwise mutual information, useful work on word embedding.

In [22]:
Pr_C = rows_sum / total_rows_sum
Pr_C[:10]

array([ 0.01000366,  0.00859214,  0.00831792,  0.00625778,  0.00621208,
        0.00438923,  0.00507302,  0.00466872,  0.00457908,  0.00389353])

In [23]:
Pr_CW = numpy.zeros(shape=(1000,5000))

In [28]:
for row in range(N_WC.shape[0]):
    for col in range(N_WC.shape[1]):
        Pr_CW[row][col] = N_WC[row][col]/np.sum(cols_sum[col])

In [ ]:
Phi_W = np.log(Pr_CW.T/Pr_C)

In [30]:
Phi_W[Phi_W < 0] = 0

In [31]:
Phi_W[np.isnan(Phi_W)] = 0

1.  Using the above information i.e metrices calculated the "(positive) pointwise mutual information" by φ(w) = max(0, log Pr(c|w) ) Pr(c)

2. We are able to represent each word embedded by context-words window of 4 words in a vector form of dimention 1000
3. Using the PCA dimensionality reduction, represented each word in V in high relevance or top  wieghted 100 dimensions. 

### Using PCA to 100-dimensional representation

In [32]:
pca = PCA(n_components=100)
Phi_W_transformed = pca.fit_transform(Phi_W)

### Using KMeans un-supervised learning algorithm to find the 100 clusters 

In [33]:
kmeans = KMeans(n_clusters=100, max_iter=300, random_state=0).fit(Phi_W_transformed)

In [98]:
j = 0
cluster_1 = []
cluster_88 = []
for cluster_num in kmeans.labels_:         #[0,0,0,0,1,2,3,100...]
    if cluster_num == 1:                 #Use Label as e.g. 90
        cluster_1.append(V[j])
    elif cluster_num == 88:
        cluster_88.append(V[j]) 
    j+=1

1. Used the KMeans un-supervised learning algorithm to find the 100 clusters of similar meaning words in Vocubulary (V)
2. As a sample print the words belong to specific cluster (e.g. Cluster = 1, 3, 5 & 88)
3.  Observed 
    - That the lenght of each cluster is different i.e. number of words in clusters are different.
    - Words in each cluster have similar or associated contextually 
    - Cluster-1 has words like -  reading, published, journal, newspapers', 'illustrated', 'survey', 'edition' etc. 
    - Cluster-88 has words like - 'lines', 'points', 'image', 'plane', 'fixed','curve', 'meets', 'pencil','tangent', 'transformed', 'arbitrary', 'transformation', 'curves', 'vertex'
4. From the sample it's obvious that dimensionlaity reduction enabled to cluster the vectors without major loss in information. As KMeans created 100 clusters which are meaning.    

cluster_1

['reading', 'published',
 'mentioned', 'experiments', 'newspaper', 'showing', 'ended', 'notes', 'publication', 'ages', 'partly',\
 'mail', 'numerous', 'collected', 'journal' 'newspapers', 'illustrated', 'survey', 'edition', 'schedule',   'visitors','latest','discussions','articles', 'marks', '1953', 'quoted', 'correspondence', 'magazines', 'weekly', 'originally', 'troubles', 'steele', 'supplement']

cluster_3

['pay',
 'industry',
 'market',
 'industrial',
 'sales',
 'farm',
 'income',
 'products',
 'demand',
 'share',
 'construction',
 'companies',
 'product',
 'capital',
 'increases',
 'potential',
 'substantial',
 'employees',
 'benefit',
 'competition',
 'budget',
 'housing',
 'vehicles',
 'raise',
 'expense',
 'shares',
 'expenditures',
 'salary',
 'investment',
 'marketing',
 'wages',
 'consumer',
 'substantially',
 'producing',
 'financing',
 'household',
 'retail',
 'earnings']

cluster_5

['company',
 'equipment',
 'food',
 'plant',
 'radio',
 'machine',
 'supply',
 'techniques',
 'materials',
 'model',
 'shelter',
 'electric',
 'electronic',
 'commercial',
 'machinery',
 'uses',
 'plants',
 'critical',
 'improved',
 'machines',
 'foods',
 'efficiency',
 'advertising',
 'manufacturers',
 'purchase',
 'supplies',
 'periods',
 'developments',
 'expensive',
 'transportation',
 'storage',
 "today's",
 'automatic',
 'tool',
 'improve',
 'handling',
 'processing',
 'foam',
 'supplied',
 'stored',
 'suitable',
 'tools',
 'quantity',
 'efficient',
 'plastic',
 'plastics',
 'drying',
 'surplus',
 "company's",
 'sba',
 'manufacturing',
 'gin',
 'manufacturer']

cluster_88

['af',
 'lines',
 'points',
 'image',
 'plane',
 'fixed',
 'follows',
 'p',
 'q',
 'curve',
 'meets',
 'pencil',
 '**zg',
 'tangent',
 'transformed',
 'arbitrary',
 'transformation',
 'curves',
 'vertex']

#### Using Nearest Neighbours for some test words using 'cosine' distance metric

In [38]:
neigh = NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=2 )

In [39]:
neigh.fit(Phi_W_transformed)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [82]:
test_words_lst = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', \
'chicago', 'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship', 'afternoon','mount', 'current', \
                 'school', 'married', 'legislators', 'voters','judges', 'million','washington' ]

In [88]:
word_neigh_dict = {}

for w in test_words_lst:
    res = neigh.kneighbors([ Phi_W_transformed[V.index(w)] ])
    ngh_word = V[ res[1][0][1] ]
    word_neigh_dict[w] = ngh_word
    
print ("Word",  " Neighbour(closest)")   
word_neigh_dict

Word  Neighbour(closest)


{'africa': 'asia',
 'afternoon': 'went',
 'autumn': 'summer',
 'chemical': 'clinical',
 'chicago': 'portland',
 'cigarette': 'lighted',
 'communism': 'danger',
 'current': 'provide',
 'detergent': 'indirect',
 'dictionary': 'text',
 'judges': 'congressional',
 'legislators': 'supervision',
 'mankind': 'world',
 'married': 'marriage',
 'million': 'billion',
 'mount': 'injured',
 'pulmonary': 'artery',
 'school': 'schools',
 'september': 'july',
 'storm': 'noon',
 'voters': 'reform',
 'washington': 'president',
 'worship': 'protestant'}

Word  Neighbour(closest)

{'africa': 'asia',
 'afternoon': 'went',
 'autumn': 'summer',
 'chemical': 'clinical',
 'chicago': 'portland',
 'cigarette': 'lighted',
 'communism': 'danger',
 'current': 'provide',
 'detergent': 'indirect',
 'dictionary': 'text',
 'judges': 'congressional',
 'legislators': 'supervision',
 'mankind': 'world',
 'married': 'marriage',
 'million': 'billion',
 'mount': 'injured',
 'pulmonary': 'artery',
 'school': 'schools',
 'september': 'july',
 'storm': 'noon',
 'voters': 'reform',
 'washington': 'president',
 'worship': 'protestant'}

1. As a final step Used the Nearest neighbor on the Reduce dimensions vectors
  - Used the Metric as 'cosine' and calculated the closest default number of neighbour as 2. Note the first closet neighbour to a given vector is the vector itself. 
  - Used a sample list of words (test_words_lst) to find the fist closet neighbour and printed the resluts. 
2. Below is the output - obviously this also makes sense (i.e. the word and it's closent semantically meaning similar word)